In [4]:
## imports
# astronomy
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
# plotting
import matplotlib.pyplot as plt
# data 
import numpy as np
import pandas as pd 

# 1. Read Data

In [5]:
# file paths 
path_catalogs = 'C:\\Users\\polar\\OneDrive - The University of Kansas\\AGNerds\\Catalogs'

In [13]:
# open tphot
tphot_data = ascii.read(path_catalogs+'\\tphot.cat')
tphot_cols = tphot_data.colnames

# show table
tphot_data.show_in_notebook()
print(tphot_cols)

# important columns:
# name = 'ra'
# name = 'dec'

['field', 'ra', 'dec', 'f560w_uJy', 'f560w_uJy_err', 'f770w_uJy', 'f770w_uJy_err', 'f1000w_uJy', 'f1000w_uJy_err', 'f1280w_uJy', 'f1280w_uJy_err', 'f1500w_uJy', 'f1500w_uJy_err', 'f1800w_uJy', 'f1800w_uJy_err', 'f2100w_uJy', 'f2100w_uJy_err']


In [11]:
# open EGS F22
egs_f22_data = Table.read(path_catalogs+'\\EGS_F22.fits.gz')
egs_f22_cols = egs_f22_data.columns

# print all columns
for col in egs_f22_cols:
    print(col)

ID
RA
DEC
CANDELS_RA
CANDELS_DEC
F606W
F814W
F105W
F125W
F140W
F160W
F36
F45
DF606W
DF814W
DF105W
DF125W
DF140W
DF160W
DF36
DF45
ZA
ZL68
ZU68
ZPEAK
ZA_NOIRAC
ZL68_NOIRAC
ZU68_NOIRAC
ZPEAK_NOIRAC


In [12]:
# open EGS FIR
egs_fir_data = Table.read(path_catalogs+'\\egs_FIR_photometry_catalog.fits')
egs_fir_cols = egs_f22_data.columns

# print all columns
for col in egs_fir_cols:
    print(col)

ID
RA
DEC
CANDELS_RA
CANDELS_DEC
F606W
F814W
F105W
F125W
F140W
F160W
F36
F45
DF606W
DF814W
DF105W
DF125W
DF140W
DF160W
DF36
DF45
ZA
ZL68
ZU68
ZPEAK
ZA_NOIRAC
ZL68_NOIRAC
ZU68_NOIRAC
ZPEAK_NOIRAC


# 2. Match RA and DEC

In [22]:
## get RA and DEC from catalogs

# get RA and DEC from tphot and make array
tphot_RA  = np.array(tphot_data['ra'])
tphot_DEC = np.array(tphot_data['dec'])
# get RA and DEC from egs_merged (already an array)
egs_f22_RA  = np.array(egs_f22_data['RA'])[0]            # ??? which egs catalog do I get RA and DEC? Are they same? 
egs_f22_DEC = np.array(egs_f22_data['DEC'])[0]

# get coordinants 
tphot_coord = SkyCoord(ra=tphot_RA*u.deg, dec=tphot_DEC*u.deg)
egs_f22_coord = SkyCoord(ra=egs_f22_RA*u.deg, dec=egs_f22_DEC*u.deg)

In [23]:
print('Number of tphot sources:\t', len(tphot_RA))
print('Number of egs_f22 sources:\t', len(egs_f22_RA))

Number of tphot sources:	 1734
Number of egs_f22 sources:	 66783


In [25]:
## match RA and DEC between catalogs
# DOC: https://docs.astropy.org/en/stable/coordinates/matchsep.html

# idx are indices into catalog that are the closest objects to each of the coordinates in c, 
# d2d are the on-sky distances between them, and 
# d3d are the 3-dimensional distances. 
idx, d2d, d3d = egs_f22_coord.match_to_catalog_sky(tphot_coord) # idx, d2d, d3d = c.match_to_catalog_sky(catalog)

# separation constraint
max_sep = 1.0 * u.arcsec
# max_sep = 0.5 * u.arcsec
sep_constraint = d2d < max_sep  # use on 'c' (egs_merged_coord)
idx_sep = idx[sep_constraint]   # use on 'catalog' (tphot)

# get matches
egs_f22_coord_matches = egs_f22_coord[sep_constraint]
tphot_coord_matches = tphot_coord[idx_sep]

# print length  
print('Number of matches:\t', len(egs_f22_coord_matches))
print('Number of matches:\t', len(tphot_coord_matches))

Number of matches:	 2026
Number of matches:	 2026


In [26]:
# test match
i=12
print(egs_f22_coord_matches[i])
print(tphot_coord_matches[i])

<SkyCoord (ICRS): (ra, dec) in deg
    (215.057096, 52.899535)>
<SkyCoord (ICRS): (ra, dec) in deg
    (215.057116, 52.89952)>
